# Libraries

In [47]:
# from sympy.physics.quantum.qubit import Qubit
# from sympy.physics.quantum.dagger import Dagger
# from sympy.physics.quantum import TensorProduct as sTP
# import sympy.physics.quantum as sq
# import sympy as sp


import qutip as qt

import cvxpy as cp
import numpy as np
import itertools




# # from multiprocessing import Pool

# #Because the arrays are large (over 4GB), they are not getting pickled (something multiprocessing needs to do)
# import pickle4reducer
# import multiprocessing as mp
# ctx = mp.get_context()
# ctx.reducer = pickle4reducer.Pickle4Reducer()



## Partial Trace for cvxpy

In [48]:
##
##https://github.com/cvxgrp/cvxpy/issues/563

import numpy as np
import cvxpy
from cvxpy.expressions.expression import Expression


def expr_as_np_array(cvx_expr):
    if cvx_expr.is_scalar():
        return np.array(cvx_expr)
    elif len(cvx_expr.shape) == 1:
        return np.array([v for v in cvx_expr])
    else:
        # then cvx_expr is a 2d array
        rows = []
        for i in range(cvx_expr.shape[0]):
            row = [cvx_expr[i,j] for j in range(cvx_expr.shape[1])]
            rows.append(row)
        arr = np.array(rows)
        return arr


def np_array_as_expr(np_arr):
    aslist = np_arr.tolist()
    expr = cvxpy.bmat(aslist)
    return expr


def np_partial_trace(rho, dims, axis=0):
    """
    Takes partial trace over the subsystem defined by 'axis'
    rho: a matrix
    dims: a list containing the dimension of each subsystem
    axis: the index of the subsytem to be traced out
    (We assume that each subsystem is square)
    """
    dims_ = np.array(dims)
    # Reshape the matrix into a tensor with the following shape:
    # [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    # Each subsystem gets one index for its row and another one for its column
    reshaped_rho = np.reshape(rho, np.concatenate((dims_, dims_), axis=None))

    # Move the subsystems to be traced towards the end
    reshaped_rho = np.moveaxis(reshaped_rho, axis, -1)
    reshaped_rho = np.moveaxis(reshaped_rho, len(dims)+axis-1, -1)

    # Trace over the very last row and column indices
    traced_out_rho = np.trace(reshaped_rho, axis1=-2, axis2=-1)

    # traced_out_rho is still in the shape of a tensor
    # Reshape back to a matrix
    dims_untraced = np.delete(dims_, axis)
    rho_dim = np.prod(dims_untraced)
    return traced_out_rho.reshape([rho_dim, rho_dim])


def partial_trace(rho, dims, axis=0,debug=False):
    if not isinstance(rho, Expression):
        rho = cvxpy.Constant(shape=rho.shape, value=rho)
    rho_np = expr_as_np_array(rho)
    traced_rho = np_partial_trace(rho_np, dims, axis)
    if(debug):
        print("this is what numpy said")
        print(traced_rho)
        for row in traced_rho:
            for element in row:
                print(element)
    ##########                
    #traced_rho = np_array_as_expr(traced_rho)
    ####################
    # if(debug):
    #     print("this is what the expression became")
    #     print(traced_rho)
    return traced_rho



## Tensor product with cvxpy

In [49]:
def general_kron(a, b):
    """
    Returns a CVXPY Expression representing the Kronecker product of a and b.
    
    At most one of "a" and "b" may be CVXPY Variable objects.
    
    :param a: 2D numpy ndarray, or a CVXPY Variable with a.ndim == 2
    :param b: 2D numpy ndarray, or a CVXPY Variable with b.ndim == 2
    """
    expr = np.kron(a, b)
    num_rows = expr.shape[0]
    rows = [cvxpy.hstack(expr[i,:]) for i in range(num_rows)]
    full_expr = cvxpy.vstack(rows)
    return full_expr

# QuTiP | DI CF with Alice testing rigidity + Weak ET

In [50]:
verbose=True
#fileName = "cache_temp_1"

## Bare Kets

In [51]:
o=qt.basis(2,0)
l=qt.basis(2,1)


I=qt.identity(2)

ι = 0+1j

TP = qt.tensor

def D(obj):
    return obj.dag()

## The unitary and the projector

### Projectors for measurement etc

In [52]:
Π_z0 = o * D(o)
Π_z1 = l * D(l)
display(Π_z0,Π_z1)

def Π_z(i):
    if i==0:
        return Π_z0
    else:
        return Π_z1


x0 = (o + l)/np.sqrt(2)
x1 = (o + l)/np.sqrt(2)


Π_x0 = x0 * D(x0)
Π_x1 = x1 * D(x1)
display(Π_x0,Π_x1)


y0 = (o + ι*l)/np.sqrt(2)
y1 = (o - ι*l)/np.sqrt(2)


Π_y0 = y0 * D(y0)
Π_y1 = y1 * D(y1)
display(Π_y0,Π_y1)


X_hat = o * D(l) + l *D(o)
display(X_hat)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. +0.5j]
 [0. -0.5j 0.5+0.j ]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

### The state, the unitary and the projectors

We define ```U, ρ1, Π``` (qutip objects), ready to use.

In [53]:
#Constructing the unitary

#XRSGYBC
#XRSG

#AA'XRSS'


#V simulates the measurement of A

V = TP(Π_x0,I,Π_z0, I,I,I) + \
    TP(Π_x1,X_hat,Π_z0, I,I,I) + \
    TP(Π_y0,I,Π_z1, I,I,I) + \
    TP(Π_y1,X_hat,Π_z1, I,I,I)

# V = TP(Π_x0,        #A(BC)
#          Π_z0,I,I, I,I,   #XYZ YZ  #ASSR(GBC)
#          I,I,I,I) + \
#       TP(Π_x1,        #A(BC)
#          Π_z0,I,I, I,I,   #XYZ YZ  #ASSR(GBC)
#          X_hat,I,I,I) + \
#       TP(Π_y0,        #A(BC)
#          Π_z1,I,I, I,I,   #XYZ YZ  #ASSR(GBC)
#          I,I,I,I) + \
#       TP(Π_y1,        #A(BC)
#          Π_z1,I,I, I,I,   #XYZ YZ  #ASSR(GBC)
#          X_hat,I,I,I)



bi = [0,1]

#To evaluate S (=S')
U = None
def f(a,x,r):
    return (a+(x*r))%2


for a,x,r in itertools.product(bi,bi,bi):        
    #print(a,x,r,f(a,x,r))
    if (f(a,x,r)):
        _ = TP(I,Π_z(a),Π_z(x),Π_z(r),X_hat,X_hat)        #AXRSS'
    else:
        _ = TP(I,Π_z(a),Π_z(x),Π_z(r),I,I)        #AXRSS'

    if U is None:
        U = _
        #display(rep(_))
        #display(_)
    else:
        U += _
        #display(_)
    #Qubit(0)
#display(U_II)

#(A)A'XRS(S')

#Measures A coherently and then trace out A (we get a mixture over the outcomes of A, stored in A')
ρ0 = (V*TP(I/2.0,o*D(o),x0*D(x0),x0*D(x0),o*D(o),o*D(o))*D(V)) #.ptrace([0,1,2,3,4,5])

#We evaluate S S' using the outcome A' and X and R and then trace out A' and S'. 
#This should yield a mixture over S. This in turn means that 

# AA'XRS(S')

ρ = (U* ρ0 *D(U)).ptrace([0,1,2,3,4])

print(ρ.tr())
display(ρ)

#display(U*D(U))
# print("Constructed the unitary")
# if(verbose): 
#     display(U)


# ρ1=ψ1.ptrace([0,3,4,5,6,7,8,9,10,11])

# Uρ1U=U*ρ1*D(U)

# #        0     12345   678   9
# #ψ1=TP(ψ_ABC,ψ_XYZYZ,ψ_ASS,ψ_R)
# #We also need to trace out S' (the first S above, i.e. index 7)
# trS_Uρ1U= Uρ1U.ptrace([0,1,2,3,4,5,6,8,9])

# if verbose:
#     print("Trace S' of this should equal trace G' σ1")
#     display(Uρ1U)



# ####################Constructing the projector

#AA'XRSGYBC

Π = None
_ = None
#GHZ winning condition
def f(x,y, b,c, r,s,g):
    if( (x+g)%2 == 1 and (s+(x*r)+b+c+1)%2 == x*y*((1+x+y)%2) ):
    # Sanity check; do I recover 0.75? It failed; TODO: revisit
    # if( (x+g)%2 == 1 ):
        return 1
    else:
        return 0

for b,c, x,y, r,s,g in itertools.product(bi,bi,bi, bi,bi,bi, bi):        
    if (f(b,c, x,y, r,s,g)==1):
        #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
        #AA'XRSGYBC
        _ = TP(I,I,Π_z(x),Π_z(r),Π_z(s),Π_z(g),Π_z(y),Π_z(b),Π_z(c))
        if Π is None:
            Π = _
            # print("calculated the symbolic part, working on the representation")
            # Π_ = rep(_)
            # print("completed the representation")
        else:
            Π += _
            # Π_ = rep(_)


display(Π)



#AA'XRSG
Π_ = None
_ = None
#GHZ winning condition
def f(x,r,s,g):
    # if( (x+g)%2 == 1 and (s+(x*r)+b+c+1)%2 == x*y*((1+x+y)%2) ):
    # Sanity check; do I recover 0.75?
    if( (x+g)%2 == 1 ):
        return 1
    else:
        return 0

for x,r,s,g in itertools.product(bi,bi,bi,bi):        
    if (f(x,r,s,g)==1):
        #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
        #XRSGYBC
        _ = TP(I,I,Π_z(x),Π_z(r),Π_z(s),Π_z(g))
        if Π_ is None:
            Π_ = _
            # print("calculated the symbolic part, working on the representation")
            # Π_ = rep(_)
            # print("completed the representation")
        else:
            Π_ += _
            # Π_ = rep(_)


display(Π)



# Π = None
# _ = None
# #GHZ winning condition
# def f(a,b,c,x,y,z,g):
#     if( (x+g)%2 == 1 and (a+b+c+1)%2 == (x*y*z)):
#         return 1
#     else:
#         return 0    
# Π_ = None
# #I_ = Π_z0 + Π_z1

# for a,b,c, x,y,z, g in itertools.product(bi,bi,bi, bi,bi,bi, bi):        
#     if (f(a,b,c, x,y,z, g)==1):
#         #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
#         _ = TP(I,        #A(BC)
#             Π_z(x), Π_z(y),Π_z(z),   #X(YZ) YZ  
#             Π_z(a),I,I,Π_z(g),Π_z(b),Π_z(c)) #A(S)SRGBC

#         if Π is None:
#             Π = _
#             # print("calculated the symbolic part, working on the representation")
#             # Π_ = rep(_)
#             # print("completed the representation")
#         else:
#             Π += _
#             # Π_ = rep(_)


# print("Constructed the projector")            
# if(verbose): 
#     display(Π)
#     # display(Π_)
#     #display(sq.represent(Π))
#     #display(rep(Π))

# #     dill.dump([U,U_I,U_II,Π], open(fileName, "wb"))
# #     print("Saved to disk")


0.9999999999999996


Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[0.0625 +0.j      0.     +0.j      0.0625 +0.j      ... 0.     +0.j
  0.03125+0.03125j 0.     +0.j     ]
 [0.     +0.j      0.     +0.j      0.     +0.j      ... 0.     +0.j
  0.     +0.j      0.     +0.j     ]
 [0.0625 +0.j      0.     +0.j      0.0625 +0.j      ... 0.     +0.j
  0.03125+0.03125j 0.     +0.j     ]
 ...
 [0.     +0.j      0.     +0.j      0.     +0.j      ... 0.0625 +0.j
  0.     +0.j      0.     +0.j     ]
 [0.03125-0.03125j 0.     +0.j      0.03125-0.03125j ... 0.     +0.j
  0.0625 +0.j      0.     +0.j     ]
 [0.     +0.j      0.     +0.j      0.     +0.j      ... 0.     +0.j
  0.     +0.j      0.     +0.j     ]]

Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2]], shape = (512, 512), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2]], shape = (512, 512), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

# The SDP itself

## Provisional implementation (doesn't quite work)

In [54]:
#should ideally be hermitian

#XRSGYBC
ρ2 = cp.Variable((2**9,2**9), hermitian=True)

#XRSG
ρ1 = cp.Variable((2**6,2**6),hermitian=True)


#constraints += [cp.trace(σ1)==1,cp.trace(σ2)==1]


#The non-trivial constraints

#So what I've done here is the following
#When converting back from numpy, it was using Vstac and Hstac
#This seemed to make everything inefficient
#So now, I just return the numpy expression for partial_trace
#and then to trace again, I use the numpy expression

trG_ρ1 = partial_trace(ρ1,[2]*6,5) #AA'XRS(G)

trC_ρ2 = partial_trace(ρ2,[2]*9,8) #AA'XRSGYB(C)
trBC_ρ2 = np_partial_trace(trC_ρ2,[2]*8,7) #AA'XRSGY(B)


# trB_σ2=partial_trace(σ2,[2]*10,8) #trace out C'; AXYZASR(B)CG
# # trBC_σ2=np_array_as_expr(np_partial_trace(trB_σ2,[2]*9,8)) #trace out C'; AXYZASR(C)G
# trBC_σ2=np_partial_trace(trB_σ2,[2]*9,8) #trace out C'; AXYZASR(C)G

# trG_σ1=partial_trace(σ1,[2]*10,9) #trace out G; AXYZYZASR(G)

# trY_σ1=partial_trace(σ1,[2]*10,2) #trace out Y; AX(Y)ZYZASRG
# # trYZ_σ1=np_array_as_expr(np_partial_trace(trY_σ1,[2]*9,2)) #trace out Z; AX(Z)YZASRG
# trYZ_σ1=np_partial_trace(trY_σ1,[2]*9,2) #trace out Z; AX(Z)YZASRG

In [ ]:
print(trBC_ρ2[0,0])

In [55]:
#constraints = [σ1>>0] #[σ1>>0, σ2>>0]
constraints = [ρ1>>0, ρ2>>0]

#Just to test, I am using the numpy expression
#And adding the condition for each element
#cvxpy wasn't accepting these numpy matrices directly

#print(trYZ_σ1[0,0])

ρ1_Y = general_kron(ρ1,np.eye(2)/2.0)



print("Working")

#BRSGY BC
n=len(trBC_ρ2) #int(len(trBC_ρ2)/(2**0))

for i in range(n):
    print(i,end='\r')
    for j in range(int(n)):
        constraints += [trBC_ρ2[i,j] == ρ1_Y[i,j]] #np.eye(n)[i,j]] #ρ1_Y[i,j]] #np.kron(ρ1,np.eye(2)/2.0)[i,j]]
print(n)
print("completed 1 of 2")


#XRS G
n=len(trG_ρ1) # int(len(trG_ρ1)/(2**0))
print("Working")
for i in range(n):
    print(i,end='\r')
    for j in range(n): #range(int(len(trG_σ1)/2)):
        constraints += [trG_ρ1[i,j] == ρ[i,j]]
print(n)
print("completed 2 of 2")




print(constraints[4])

# constraints += [trG_σ1 == trS_Uρ1U.data, trBC_σ2 == trYZ_σ1 ]

#constraints += [ cp.trace(σ1) == cp.trace(trS_Uρ1U),cp.trace(σ2)==cp.trace(σ1) ]


Working
128
completed 1 of 2
Working
32
completed 2 of 2
var559240[0, 8] + var559240[1, 9] + var559240[2, 10] + var559240[3, 11] == Vstack(Hstack(var559241 @ Promote(0.5, (64, 64)), var559241 @ Promote(0.0, (64, 64))), Hstack(var559241 @ Promote(0.0, (64, 64)), var559241 @ Promote(0.5, (64, 64))))[0, 2]


In [ ]:
#trG_σ1
#trS_Uρ1U.data
#print(trBC_σ2[0,0])

In [ ]:
#trYZ_σ1[1,1]

In [56]:
prob = cp.Problem(cp.Maximize(cp.trace(cp.real(Π.data @ ρ2 ))),constraints)
#prob = cp.Problem(cp.Maximize(cp.trace(cp.real(Π_.data @ ρ1 ))),constraints)

prob.solve(solver=cp.SCS,verbose=True) #,max_iters=10000) #MOSEK or SCS


WARN: A->p (column pointers) not strictly increasing, column 131328 empty
WARN: A->p (column pointers) not strictly increasing, column 131841 empty
WARN: A->p (column pointers) not strictly increasing, column 132354 empty
WARN: A->p (column pointers) not strictly increasing, column 132867 empty
WARN: A->p (column pointers) not strictly increasing, column 133380 empty
WARN: A->p (column pointers) not strictly increasing, column 133893 empty
WARN: A->p (column pointers) not strictly increasing, column 134406 empty
WARN: A->p (column pointers) not strictly increasing, column 134919 empty
WARN: A->p (column pointers) not strictly increasing, column 135432 empty
WARN: A->p (column pointers) not strictly increasing, column 135945 empty
WARN: A->p (column pointers) not strictly increasing, column 136458 empty
WARN: A->p (column pointers) not strictly increasing, column 136971 empty
WARN: A->p (column pointers) not strictly increasing, column 137484 empty
WARN: A->p (column pointers) not stric

Setup time: 1.18e+00s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.67e+21  8.08e+21  1.00e+00 -1.54e+22  3.22e+22  9.25e+21  5.53e-01 
    60| 1.83e-07  7.08e-07  1.61e-07 -7.50e-01 -7.50e-01  6.91e-17  3.63e+01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.64e+01s
	Lin-sys: nnz in L factor: 2142688, avg solve time: 1.63e-02s
	Cones: avg projection time: 4.91e-01s
	Acceleration: avg step time: 6.17e-02s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.1065e-09, dist(y, K*) = 2.4103e-09, s'y/|s||y| = 2.6815e-13
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 1.8318e-07
dual res:   |A'y + c|_2 / (1 + |c|_2) = 7.0850e-07
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 1.6094e-

0.749999780468343

In [ ]:
# prob = cp.Problem(cp.Maximize(cp.trace(cp.real(Π.data @ ρ2 ))),constraints)
prob = cp.Problem(cp.Maximize(cp.trace(cp.real(Π_.data @ ρ1 ))),constraints)

prob.solve(solver=cp.SCS,verbose=True)#,max_iters=10000) #MOSEK or SCS


In [ ]:
Post Jamie Magic, Debug (SCS)


----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 63632
eps = 1.00e-04, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 25032, constraints m = 35600
Cones:	primal zero / dual free vars: 2176
	sd vars: 33424, sd blks: 2
Setup time: 2.94e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.78e+20  1.93e+21  1.00e+00 -1.77e+21  3.87e+21  1.18e+21  2.19e-02 
   100| 2.84e-03  1.14e-02  7.25e-04 -5.01e-01 -5.03e-01  1.64e-16  1.79e+00 
   200| 9.53e-04  4.12e-03  1.62e-03 -5.02e-01 -4.99e-01  3.99e-17  3.60e+00 
   300| 1.44e-03  5.00e-03  9.45e-06 -4.95e-01 -4.95e-01  2.79e-16  5.41e+00 
   400| 6.24e-04  2.80e-03  1.65e-04 -4.95e-01 -4.95e-01  3.75e-16  7.25e+00 
   500| 5.50e-04  2.11e-03  3.97e-05 -4.94e-01 -4.94e-01  1.63e-16  9.16e+00 
   600| 7.81e-04  2.34e-03  8.30e-05 -4.92e-01 -4.92e-01  1.76e-16  1.10e+01 
   700| 1.86e-02  7.48e-02  4.89e-04 -4.94e-01 -4.95e-01  3.77e-16  1.29e+01 
   800| 8.47e-02  3.46e-01  1.70e-03 -5.04e-01 -5.00e-01  1.98e-17  1.48e+01 
   900| 4.02e-04  1.87e-03  2.89e-04 -4.93e-01 -4.94e-01  3.66e-16  1.67e+01 
  1000| 2.66e-04  1.12e-03  1.20e-04 -4.94e-01 -4.93e-01  1.83e-16  1.85e+01 
  1100| 3.70e-04  1.41e-03  1.27e-04 -4.91e-01 -4.91e-01  2.31e-16  2.04e+01 
  1200| 6.12e-04  3.00e-03  4.15e-04 -4.92e-01 -4.93e-01  1.58e-16  2.22e+01 
  1300| 1.51e-04  6.16e-04  6.43e-07 -4.92e-01 -4.92e-01  1.01e-16  2.41e+01 
  1400| 4.67e-04  1.96e-03  1.33e-05 -4.93e-01 -4.93e-01  1.46e-16  2.60e+01 
  1500| 2.44e-04  1.10e-03  5.48e-05 -4.92e-01 -4.92e-01  4.04e-17  2.78e+01 
  1600| 4.86e-04  1.98e-03  1.69e-04 -4.92e-01 -4.92e-01  1.51e-16  2.97e+01 
  1700| 6.06e-04  3.37e-03  3.81e-04 -4.92e-01 -4.93e-01  2.20e-16  3.15e+01 
  1800| 3.57e-04  1.32e-03  7.41e-05 -4.92e-01 -4.92e-01  1.79e-16  3.34e+01 
  1900| 2.17e-04  9.43e-04  3.50e-05 -4.92e-01 -4.92e-01  3.32e-16  3.53e+01 
  2000| 2.65e-04  1.32e-03  1.04e-05 -4.92e-01 -4.92e-01  5.58e-17  3.71e+01 
  2100| 1.52e-04  6.23e-04  4.88e-05 -4.92e-01 -4.92e-01  1.04e-16  3.90e+01 
  2200| 1.19e-04  4.92e-04  2.54e-05 -4.92e-01 -4.92e-01  1.31e-16  4.08e+01 
  2300| 2.57e-04  9.42e-04  3.70e-05 -4.91e-01 -4.91e-01  1.79e-16  4.27e+01 
  2400| 3.90e-04  1.41e-03  2.27e-05 -4.92e-01 -4.92e-01  1.21e-16  4.45e+01 
  2500| 1.14e-04  4.00e-04  4.03e-05 -4.92e-01 -4.92e-01  2.51e-16  4.64e+01 
  2600| 2.23e-04  8.11e-04  7.23e-05 -4.92e-01 -4.92e-01  1.41e-16  4.82e+01 
  2700| 3.45e-02  1.44e-01  1.56e-03 -4.80e-01 -4.77e-01  9.40e-17  5.01e+01 
  2800| 5.26e-05  2.02e-04  3.85e-05 -4.92e-01 -4.92e-01  6.19e-17  5.20e+01 
  2900| 2.86e-04  1.02e-03  1.70e-05 -4.92e-01 -4.92e-01  3.73e-17  5.38e+01 
  3000| 1.88e-04  7.89e-04  5.50e-05 -4.92e-01 -4.92e-01  3.47e-17  5.57e+01 
  3100| 2.94e-04  1.08e-03  1.08e-04 -4.91e-01 -4.91e-01  3.53e-17  5.75e+01 
  3200| 1.39e-04  5.55e-04  5.76e-05 -4.92e-01 -4.92e-01  1.04e-17  5.94e+01 
  3300| 6.65e-04  2.80e-03  6.91e-05 -4.92e-01 -4.92e-01  5.57e-17  6.12e+01 
  3400| 2.87e-04  1.15e-03  5.58e-05 -4.92e-01 -4.91e-01  1.12e-16  6.31e+01 
  3500| 9.90e-05  4.80e-04  4.87e-05 -4.92e-01 -4.92e-01  1.08e-17  6.49e+01 
  3600| 2.46e-04  9.68e-04  1.29e-05 -4.92e-01 -4.92e-01  2.86e-16  6.68e+01 
  3700| 6.99e-05  2.44e-04  5.97e-05 -4.92e-01 -4.92e-01  8.94e-17  6.86e+01 
  3800| 4.57e-03  2.05e-02  1.33e-04 -4.92e-01 -4.93e-01  1.03e-16  7.04e+01 
  3900| 9.99e-05  3.60e-04  2.52e-05 -4.92e-01 -4.92e-01  2.09e-16  7.23e+01 
  4000| 2.02e-03  7.52e-03  8.41e-05 -4.92e-01 -4.92e-01  8.10e-17  7.41e+01 
  4100| 6.37e-05  2.81e-04  6.20e-05 -4.92e-01 -4.92e-01  2.46e-17  7.60e+01 
  4200| 1.13e-04  3.90e-04  1.27e-04 -4.92e-01 -4.92e-01  1.85e-16  7.78e+01 
  4300| 7.76e-05  3.70e-04  7.71e-05 -4.92e-01 -4.91e-01  6.18e-17  7.97e+01 
  4400| 2.19e-02  8.73e-02  6.76e-05 -4.93e-01 -4.93e-01  1.55e-16  8.15e+01 
  4500| 2.58e-04  1.19e-03  3.42e-05 -4.92e-01 -4.92e-01  2.37e-16  8.33e+01 
  4600| 8.77e-05  3.69e-04  7.33e-05 -4.92e-01 -4.92e-01  2.74e-17  8.52e+01 
  4700| 1.31e-03  5.41e-03  1.94e-04 -4.91e-01 -4.91e-01  1.34e-16  8.70e+01 
  4800| 6.74e-05  3.13e-04  6.05e-05 -4.92e-01 -4.92e-01  2.49e-16  8.88e+01 
  4900| 7.26e-05  2.91e-04  1.29e-05 -4.92e-01 -4.92e-01  2.06e-16  9.07e+01 
  5000| 5.13e-05  1.99e-04  4.41e-05 -4.92e-01 -4.92e-01  2.56e-16  9.25e+01 
  5020| 2.57e-05  9.43e-05  2.03e-05 -4.92e-01 -4.92e-01  7.74e-17  9.29e+01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 9.29e+01s
	Lin-sys: nnz in L factor: 133624, avg solve time: 6.54e-04s
	Cones: avg projection time: 1.29e-02s
	Acceleration: avg step time: 3.15e-03s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.8391e-09, dist(y, K*) = 2.8217e-09, s'y/|s||y| = -6.7585e-11
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 2.5726e-05
dual res:   |A'y + c|_2 / (1 + |c|_2) = 9.4316e-05
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 2.0331e-05
----------------------------------------------------------------------------
c'x = -0.4917, -b'y = -0.4917
============================================================================
0.4917360927526637







----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 63632
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 25032, constraints m = 35600
Cones:	primal zero / dual free vars: 2176
	sd vars: 33424, sd blks: 2
Setup time: 1.29e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.78e+20  1.93e+21  1.00e+00 -1.77e+21  3.87e+21  1.18e+21  5.28e-02 
   100| 2.84e-03  1.14e-02  7.25e-04 -5.01e-01 -5.03e-01  1.64e-16  1.90e+00 
   200| 9.53e-04  4.12e-03  1.62e-03 -5.02e-01 -4.99e-01  3.99e-17  3.82e+00 
   300| 1.44e-03  5.00e-03  9.45e-06 -4.95e-01 -4.95e-01  2.79e-16  5.67e+00 
   400| 6.24e-04  2.80e-03  1.65e-04 -4.95e-01 -4.95e-01  3.75e-16  7.54e+00 
   500| 5.50e-04  2.11e-03  3.97e-05 -4.94e-01 -4.94e-01  1.63e-16  9.40e+00 
   600| 7.81e-04  2.34e-03  8.30e-05 -4.92e-01 -4.92e-01  1.76e-16  1.13e+01 
   700| 1.86e-02  7.48e-02  4.89e-04 -4.94e-01 -4.95e-01  3.77e-16  1.32e+01 
   800| 8.47e-02  3.46e-01  1.70e-03 -5.04e-01 -5.00e-01  1.98e-17  1.50e+01 
   900| 4.02e-04  1.87e-03  2.89e-04 -4.93e-01 -4.94e-01  3.66e-16  1.69e+01 
  1000| 2.66e-04  1.12e-03  1.20e-04 -4.94e-01 -4.93e-01  1.83e-16  1.88e+01 
  1100| 3.70e-04  1.41e-03  1.27e-04 -4.91e-01 -4.91e-01  2.31e-16  2.07e+01 
  1200| 6.12e-04  3.00e-03  4.15e-04 -4.92e-01 -4.93e-01  1.58e-16  2.26e+01 
  1300| 1.51e-04  6.16e-04  6.43e-07 -4.92e-01 -4.92e-01  1.01e-16  2.44e+01 
  1400| 4.67e-04  1.96e-03  1.33e-05 -4.93e-01 -4.93e-01  1.46e-16  2.63e+01 
  1500| 2.44e-04  1.10e-03  5.48e-05 -4.92e-01 -4.92e-01  4.04e-17  2.82e+01 
  1600| 4.86e-04  1.98e-03  1.69e-04 -4.92e-01 -4.92e-01  1.51e-16  3.01e+01 
  1700| 6.06e-04  3.37e-03  3.81e-04 -4.92e-01 -4.93e-01  2.20e-16  3.19e+01 
  1800| 3.57e-04  1.32e-03  7.41e-05 -4.92e-01 -4.92e-01  1.79e-16  3.38e+01 
  1900| 2.17e-04  9.43e-04  3.50e-05 -4.92e-01 -4.92e-01  3.32e-16  3.57e+01 
  2000| 2.65e-04  1.32e-03  1.04e-05 -4.92e-01 -4.92e-01  5.58e-17  3.76e+01 
  2100| 1.52e-04  6.23e-04  4.88e-05 -4.92e-01 -4.92e-01  1.04e-16  3.94e+01 
  2200| 1.19e-04  4.92e-04  2.54e-05 -4.92e-01 -4.92e-01  1.31e-16  4.13e+01 
  2300| 2.57e-04  9.42e-04  3.70e-05 -4.91e-01 -4.91e-01  1.79e-16  4.32e+01 
  2400| 3.90e-04  1.41e-03  2.27e-05 -4.92e-01 -4.92e-01  1.21e-16  4.50e+01 
  2500| 1.14e-04  4.00e-04  4.03e-05 -4.92e-01 -4.92e-01  2.51e-16  4.69e+01 
  2600| 2.23e-04  8.11e-04  7.23e-05 -4.92e-01 -4.92e-01  1.41e-16  4.87e+01 
  2700| 3.45e-02  1.44e-01  1.56e-03 -4.80e-01 -4.77e-01  9.40e-17  5.06e+01 
  2800| 5.26e-05  2.02e-04  3.85e-05 -4.92e-01 -4.92e-01  6.19e-17  5.25e+01 
  2900| 2.86e-04  1.02e-03  1.70e-05 -4.92e-01 -4.92e-01  3.73e-17  5.43e+01 
  3000| 1.88e-04  7.89e-04  5.50e-05 -4.92e-01 -4.92e-01  3.47e-17  5.62e+01 
  3100| 2.94e-04  1.08e-03  1.08e-04 -4.91e-01 -4.91e-01  3.53e-17  5.81e+01 
  3200| 1.39e-04  5.55e-04  5.76e-05 -4.92e-01 -4.92e-01  1.04e-17  5.99e+01 
  3300| 6.65e-04  2.80e-03  6.91e-05 -4.92e-01 -4.92e-01  5.57e-17  6.18e+01 
  3400| 2.87e-04  1.15e-03  5.58e-05 -4.92e-01 -4.91e-01  1.12e-16  6.36e+01 
  3500| 9.90e-05  4.80e-04  4.87e-05 -4.92e-01 -4.92e-01  1.08e-17  6.55e+01 
  3600| 2.46e-04  9.68e-04  1.29e-05 -4.92e-01 -4.92e-01  2.86e-16  6.74e+01 
  3700| 6.99e-05  2.44e-04  5.97e-05 -4.92e-01 -4.92e-01  8.94e-17  6.92e+01 
  3800| 4.57e-03  2.05e-02  1.33e-04 -4.92e-01 -4.93e-01  1.03e-16  7.11e+01 
  3900| 9.99e-05  3.60e-04  2.52e-05 -4.92e-01 -4.92e-01  2.09e-16  7.29e+01 
  4000| 2.02e-03  7.52e-03  8.41e-05 -4.92e-01 -4.92e-01  8.10e-17  7.48e+01 
  4100| 6.37e-05  2.81e-04  6.20e-05 -4.92e-01 -4.92e-01  2.46e-17  7.66e+01 
  4200| 1.13e-04  3.90e-04  1.27e-04 -4.92e-01 -4.92e-01  1.85e-16  7.85e+01 
  4300| 7.76e-05  3.70e-04  7.71e-05 -4.92e-01 -4.91e-01  6.18e-17  8.03e+01 
  4400| 2.19e-02  8.73e-02  6.76e-05 -4.93e-01 -4.93e-01  1.55e-16  8.21e+01 
  4500| 2.58e-04  1.19e-03  3.42e-05 -4.92e-01 -4.92e-01  2.37e-16  8.40e+01 
  4600| 8.77e-05  3.69e-04  7.33e-05 -4.92e-01 -4.92e-01  2.74e-17  8.58e+01 
  4700| 1.31e-03  5.41e-03  1.94e-04 -4.91e-01 -4.91e-01  1.34e-16  8.77e+01 
  4800| 6.74e-05  3.13e-04  6.05e-05 -4.92e-01 -4.92e-01  2.49e-16  8.95e+01 
  4900| 7.26e-05  2.91e-04  1.29e-05 -4.92e-01 -4.92e-01  2.06e-16  9.14e+01 
  5000| 1.26e-02  5.36e-02  1.20e-04 -4.92e-01 -4.92e-01  1.60e-16  9.32e+01 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate, returning best found solution.
Timing: Solve time: 9.32e+01s
	Lin-sys: nnz in L factor: 133624, avg solve time: 6.41e-04s
	Cones: avg projection time: 1.30e-02s
	Acceleration: avg step time: 3.15e-03s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.6996e-09, dist(y, K*) = 3.2270e-09, s'y/|s||y| = -3.5626e-10
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 3.6811e-05
dual res:   |A'y + c|_2 / (1 + |c|_2) = 1.1034e-04
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 2.8141e-05
----------------------------------------------------------------------------
c'x = -0.4916, -b'y = -0.4917
============================================================================




----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 63632
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 25032, constraints m = 35600
Cones:	primal zero / dual free vars: 2176
	sd vars: 33424, sd blks: 2
Setup time: 3.33e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.78e+20  1.93e+21  1.00e+00 -1.77e+21  3.87e+21  1.18e+21  1.81e-02 
   100| 1.42e-03  4.81e-03  3.19e-03 -5.33e-01 -5.26e-01  1.39e-16  1.54e+00 
   200| 8.36e-04  3.09e-03  2.05e-03 -5.18e-01 -5.14e-01  1.93e-16  3.05e+00 
   300| 6.08e-04  2.33e-03  1.54e-03 -5.12e-01 -5.09e-01  1.21e-16  4.56e+00 
   400| 4.83e-04  1.88e-03  1.25e-03 -5.08e-01 -5.06e-01  3.31e-21  6.07e+00 
   500| 4.03e-04  1.59e-03  1.05e-03 -5.06e-01 -5.03e-01  1.69e-16  7.58e+00 
   600| 3.47e-04  1.38e-03  9.13e-04 -5.04e-01 -5.02e-01  1.73e-21  9.09e+00 
   700| 3.05e-04  1.22e-03  8.07e-04 -5.02e-01 -5.01e-01  2.14e-16  1.06e+01 
   800| 2.73e-04  1.09e-03  7.24e-04 -5.01e-01 -5.00e-01  1.18e-16  1.21e+01 
   900| 2.47e-04  9.90e-04  6.57e-04 -5.00e-01 -4.99e-01  2.49e-23  1.36e+01 
  1000| 2.26e-04  9.07e-04  6.02e-04 -5.00e-01 -4.98e-01  1.40e-16  1.51e+01 
  1100| 2.08e-04  8.37e-04  5.55e-04 -4.99e-01 -4.98e-01  1.51e-16  1.66e+01 
  1200| 1.93e-04  7.77e-04  5.16e-04 -4.99e-01 -4.97e-01  1.62e-16  1.81e+01 
  1300| 1.80e-04  7.25e-04  4.82e-04 -4.98e-01 -4.97e-01  1.73e-16  1.96e+01 
  1400| 1.69e-04  6.80e-04  4.52e-04 -4.98e-01 -4.97e-01  2.20e-22  2.11e+01 
  1500| 1.59e-04  6.41e-04  4.25e-04 -4.97e-01 -4.96e-01  1.39e-25  2.26e+01 
  1600| 1.50e-04  6.06e-04  4.02e-04 -4.97e-01 -4.96e-01  9.09e-23  2.41e+01 
  1700| 1.42e-04  5.74e-04  3.81e-04 -4.97e-01 -4.96e-01  8.92e-25  2.56e+01 
  1800| 1.35e-04  5.46e-04  3.62e-04 -4.96e-01 -4.96e-01  1.87e-24  2.71e+01 
  1900| 1.29e-04  5.21e-04  3.46e-04 -4.96e-01 -4.96e-01  1.19e-16  2.86e+01 
  2000| 1.23e-04  4.97e-04  3.30e-04 -4.96e-01 -4.95e-01  1.24e-16  3.01e+01 
  2100| 1.18e-04  4.76e-04  3.16e-04 -4.96e-01 -4.95e-01  1.29e-16  3.16e+01 
  2200| 1.13e-04  4.57e-04  3.03e-04 -4.96e-01 -4.95e-01  1.35e-16  3.31e+01 
  2300| 1.08e-04  4.39e-04  2.91e-04 -4.96e-01 -4.95e-01  1.40e-16  3.46e+01 
  2400| 1.04e-04  4.22e-04  2.80e-04 -4.95e-01 -4.95e-01  4.31e-23  3.61e+01 
  2500| 1.00e-04  4.07e-04  2.70e-04 -4.95e-01 -4.95e-01  1.51e-16  3.76e+01 
  2600| 9.68e-05  3.93e-04  2.61e-04 -4.95e-01 -4.95e-01  3.12e-16  3.91e+01 
  2700| 9.35e-05  3.79e-04  2.52e-04 -4.95e-01 -4.95e-01  1.61e-16  4.06e+01 
  2800| 9.05e-05  3.67e-04  2.44e-04 -4.95e-01 -4.94e-01  1.67e-16  4.21e+01 
  2900| 8.76e-05  3.56e-04  2.36e-04 -4.95e-01 -4.94e-01  1.72e-16  4.36e+01 
  3000| 8.49e-05  3.45e-04  2.29e-04 -4.95e-01 -4.94e-01  1.77e-16  4.51e+01 
  3100| 8.24e-05  3.34e-04  2.22e-04 -4.95e-01 -4.94e-01  1.38e-23  4.67e+01 
  3200| 8.00e-05  3.25e-04  2.16e-04 -4.95e-01 -4.94e-01  1.88e-16  4.82e+01 
  3300| 7.78e-05  3.16e-04  2.10e-04 -4.94e-01 -4.94e-01  1.93e-16  4.96e+01 
  3400| 7.56e-05  3.07e-04  2.04e-04 -4.94e-01 -4.94e-01  2.32e-23  5.11e+01 
  3500| 7.36e-05  2.99e-04  1.99e-04 -4.94e-01 -4.94e-01  4.07e-16  5.26e+01 
  3600| 7.17e-05  2.91e-04  1.93e-04 -4.94e-01 -4.94e-01  2.09e-16  5.41e+01 
  3700| 6.99e-05  2.84e-04  1.89e-04 -4.94e-01 -4.94e-01  2.14e-16  5.56e+01 
  3800| 6.82e-05  2.77e-04  1.84e-04 -4.94e-01 -4.94e-01  2.83e-23  5.71e+01 
  3900| 6.66e-05  2.71e-04  1.80e-04 -4.94e-01 -4.94e-01  9.19e-26  5.86e+01 
  4000| 6.50e-05  2.64e-04  1.75e-04 -4.94e-01 -4.94e-01  6.64e-26  6.01e+01 
  4100| 6.36e-05  2.58e-04  1.71e-04 -4.94e-01 -4.94e-01  1.17e-16  6.16e+01 
  4200| 6.21e-05  2.53e-04  1.68e-04 -4.94e-01 -4.94e-01  1.20e-16  6.31e+01 
  4300| 6.08e-05  2.47e-04  1.64e-04 -4.94e-01 -4.94e-01  3.75e-24  6.46e+01 
  4400| 5.95e-05  2.42e-04  1.60e-04 -4.94e-01 -4.93e-01  1.25e-16  6.61e+01 
  4500| 5.83e-05  2.37e-04  1.57e-04 -4.94e-01 -4.93e-01  1.28e-16  6.76e+01 
  4600| 5.71e-05  2.32e-04  1.54e-04 -4.94e-01 -4.93e-01  2.61e-16  6.91e+01 
  4700| 5.59e-05  2.27e-04  1.51e-04 -4.94e-01 -4.93e-01  1.33e-16  7.06e+01 
  4800| 5.48e-05  2.23e-04  1.48e-04 -4.94e-01 -4.93e-01  1.36e-16  7.21e+01 
  4900| 5.38e-05  2.19e-04  1.45e-04 -4.94e-01 -4.93e-01  8.77e-26  7.36e+01 
  5000| 5.28e-05  2.15e-04  1.42e-04 -4.94e-01 -4.93e-01  5.32e-26  7.50e+01 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate, returning best found solution.
Timing: Solve time: 7.50e+01s
	Lin-sys: nnz in L factor: 133624, avg solve time: 5.96e-04s
	Cones: avg projection time: 1.28e-02s
	Acceleration: avg step time: 1.66e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.0300e-09, dist(y, K*) = 3.7683e-09, s'y/|s||y| = 4.0638e-11
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 5.2767e-05
dual res:   |A'y + c|_2 / (1 + |c|_2) = 2.1454e-04
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 1.4238e-04
----------------------------------------------------------------------------
c'x = -0.4935, -b'y = -0.4933
============================================================================


In [ ]:
Post Jamie Magic #1

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 63632
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 25032, constraints m = 35600
Cones:	primal zero / dual free vars: 2176
	sd vars: 33424, sd blks: 2
Setup time: 1.07e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.62e+20  2.03e+21  1.00e+00 -2.34e+21  5.02e+21  1.47e+21  4.72e-02 
   100| 1.68e-03  7.53e-03  4.59e-04 -4.36e-01 -4.35e-01  6.35e-17  1.95e+00 
   200| 2.22e-03  7.93e-03  7.24e-04 -4.40e-01 -4.42e-01  1.56e-16  3.92e+00 
   300| 4.12e-04  1.46e-03  5.52e-04 -4.38e-01 -4.37e-01  2.20e-18  5.91e+00 
   400| 3.97e+17  1.28e+18  1.55e-03  2.71e+17  2.70e+17  1.68e+18  7.87e+00 
   500| 1.00e-04  3.98e-04  2.32e-04 -4.38e-01 -4.38e-01  4.24e-16  9.78e+00 
   600| 9.82e-04  3.21e-03  2.12e-03 -4.37e-01 -4.41e-01  2.50e-16  1.17e+01 
   700| 6.53e-03  2.07e-02  1.47e-02 -4.34e-01 -4.62e-01  1.53e-16  1.35e+01 
   800| 3.59e-04  1.15e-03  7.86e-04 -4.37e-01 -4.39e-01  3.79e-16  1.54e+01 
   900| 3.98e-04  1.39e-03  6.97e-04 -4.37e-01 -4.39e-01  2.41e-16  1.73e+01 
  1000| 2.23e-03  7.10e-03  4.83e-03 -4.36e-01 -4.45e-01  2.59e-16  1.92e+01 
  1100| 1.67e+15  5.39e+15  3.88e-03  1.14e+15  1.13e+15  7.03e+15  2.11e+01 
  1200| 2.99e+15  9.63e+15  1.99e-03  2.03e+15  2.02e+15  1.25e+16  2.30e+01 
  1300| 5.01e-02  1.71e-01  8.12e-04 -4.41e-01 -4.43e-01  9.25e-17  2.50e+01 
  1400| 8.06e-04  2.62e-03  1.76e-03 -4.37e-01 -4.40e-01  3.34e-18  2.69e+01 
  1500| 1.95e-03  6.39e-03  4.55e-03 -4.37e-01 -4.46e-01  1.31e-16  2.88e+01 
  1600| 1.43e+15  4.60e+15  1.82e-03  9.44e+14  9.41e+14  5.87e+15  3.07e+01 
  1700| 2.97e+15  9.55e+15  1.81e-03  2.02e+15  2.01e+15  1.25e+16  3.26e+01 
  1800| 3.51e-04  1.15e-03  7.18e-04 -4.38e-01 -4.39e-01  1.66e-16  3.46e+01 
  1900| 7.83e-04  2.55e-03  1.67e-03 -4.37e-01 -4.40e-01  2.19e-16  3.65e+01 
  2000| 1.75e-03  5.44e-03  4.20e-03 -4.37e-01 -4.45e-01  4.12e-16  3.84e+01 
  2100| 2.21e-03  7.12e-03  4.93e-03 -4.36e-01 -4.45e-01  3.74e-17  4.03e+01 
  2200| 9.27e-04  3.00e-03  2.06e-03 -4.37e-01 -4.41e-01  3.82e-16  4.22e+01 
  2300| 8.87e-04  2.66e-03  2.33e-03 -4.37e-01 -4.41e-01  3.95e-16  4.41e+01 
  2400| 4.75e-04  1.50e-03  8.38e-04 -4.37e-01 -4.38e-01  6.16e-17  4.61e+01 
  2500| 6.44e-04  2.15e-03  1.30e-03 -4.37e-01 -4.40e-01  1.62e-16  4.80e+01 
  2600| 3.90e-02  1.39e-01  8.03e-04 -4.39e-01 -4.38e-01  2.65e-16  4.99e+01 
  2700| 6.06e-04  1.93e-03  1.39e-03 -4.37e-01 -4.40e-01  1.98e-16  5.18e+01 
  2800| 8.83e-04  2.89e-03  1.85e-03 -4.37e-01 -4.40e-01  6.59e-17  5.37e+01 
  2900| 5.84e-05  2.52e-04  1.36e-05 -4.37e-01 -4.37e-01  1.27e-16  5.57e+01 
  3000| 4.24e+15  1.37e+16  8.99e-04  2.81e+15  2.82e+15  1.75e+16  5.76e+01 
  3100| 7.00e-05  2.10e-04  5.89e-05 -4.37e-01 -4.38e-01  4.36e-16  5.96e+01 
  3160| 2.50e-05  6.26e-05  4.69e-06 -4.38e-01 -4.38e-01  6.04e-17  6.08e+01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 6.08e+01s
	Lin-sys: nnz in L factor: 133624, avg solve time: 6.50e-04s
	Cones: avg projection time: 1.36e-02s
	Acceleration: avg step time: 3.16e-03s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 7.5980e-09, dist(y, K*) = 3.1513e-09, s'y/|s||y| = -2.0472e-10
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 2.4997e-05
dual res:   |A'y + c|_2 / (1 + |c|_2) = 6.2568e-05
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 4.6943e-06
----------------------------------------------------------------------------
c'x = -0.4375, -b'y = -0.4375
============================================================================
0.4375148594226565

# Temporary

In [ ]:
#Uttam Bhai's formula
#उत्तम भाई

for i in [True,False]:
    for j in [True,False]:
        for k in [True,False]:
            print ((i or (j and k)), ((i or j) and (i or k)))            